<a href="https://colab.research.google.com/github/SatyamKumarCS/ML_Models/blob/main/IDMB_Feedback.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
import numpy as np
import keras
from tensorflow.keras.datasets import imdb
from keras import models,layers
from keras.src.utils import pad_sequences
from warnings import filterwarnings
filterwarnings('ignore')

In [56]:
(xtrain,ytrain),(xtest,ytest)=imdb.load_data()

In [57]:
imdb.get_word_index()

{'fawn': 34701,
 'tsukino': 52006,
 'nunnery': 52007,
 'sonja': 16816,
 'vani': 63951,
 'woods': 1408,
 'spiders': 16115,
 'hanging': 2345,
 'woody': 2289,
 'trawling': 52008,
 "hold's": 52009,
 'comically': 11307,
 'localized': 40830,
 'disobeying': 30568,
 "'royale": 52010,
 "harpo's": 40831,
 'canet': 52011,
 'aileen': 19313,
 'acurately': 52012,
 "diplomat's": 52013,
 'rickman': 25242,
 'arranged': 6746,
 'rumbustious': 52014,
 'familiarness': 52015,
 "spider'": 52016,
 'hahahah': 68804,
 "wood'": 52017,
 'transvestism': 40833,
 "hangin'": 34702,
 'bringing': 2338,
 'seamier': 40834,
 'wooded': 34703,
 'bravora': 52018,
 'grueling': 16817,
 'wooden': 1636,
 'wednesday': 16818,
 "'prix": 52019,
 'altagracia': 34704,
 'circuitry': 52020,
 'crotch': 11585,
 'busybody': 57766,
 "tart'n'tangy": 52021,
 'burgade': 14129,
 'thrace': 52023,
 "tom's": 11038,
 'snuggles': 52025,
 'francesco': 29114,
 'complainers': 52027,
 'templarios': 52125,
 '272': 40835,
 '273': 52028,
 'zaniacs': 52130,

In [58]:
print(max(len(seq) for seq in xtrain))
print(max(len(seq) for seq in xtest))

2494
2315


In [59]:
xtrainP=pad_sequences(xtrain,padding='pre',maxlen=2494)
xtestP=pad_sequences(xtest,padding='pre',maxlen=2494)
xtestP.shape

(25000, 2494)

In [60]:
print(len(np.unique(xtrainP)))

88586


In [61]:
imdb_model=models.Sequential()

In [62]:
imdb_model.add(
    keras.layers.Embedding(
        input_dim=88586,
        output_dim=15,
        input_length=2494
    )
)

In [63]:
imdb_model.add(keras.layers.Flatten())
imdb_model.add(layers.Dense(8))
imdb_model.add(layers.Dense(1,activation="sigmoid"))

In [64]:
imdb_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [65]:
imdb_model.fit(xtrainP, ytrain_tf, epochs=10, validation_data=(xtestP, ytest_tf))

Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - accuracy: 0.6962 - loss: 0.5733 - val_accuracy: 0.8674 - val_loss: 0.3094
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9504 - loss: 0.1453 - val_accuracy: 0.8739 - val_loss: 0.3139
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.9941 - loss: 0.0316 - val_accuracy: 0.8445 - val_loss: 0.5081
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.9992 - loss: 0.0060 - val_accuracy: 0.8685 - val_loss: 0.4520
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 1.0000 - loss: 0.0013 - val_accuracy: 0.8682 - val_loss: 0.4876
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 1.0000 - loss: 4.4302e-04 - val_accuracy: 0.8687 - val_loss: 0.5151
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 1.0000 - loss: 2.4143e-04 - val_accuracy: 0.8676 - val_loss: 0.5478
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 1.0000 - loss: 1.3294e-04 - val_

In [67]:
import numpy as np

# 1. Extract the 15D map from the Embedding layer
embedding_matrix = imdb_model.layers[0].get_weights()[0]

# 2. Build a reverse dictionary (ID -> Word)
word_index = imdb.get_word_index()
reverse_word_index = {value + 3: key for key, value in word_index.items()}
reverse_word_index[0] = "<PAD>"
reverse_word_index[1] = "<START>"
reverse_word_index[2] = "<UNKNOWN>"

def find_closest_words(target_word, top_n=5):
  # 3. Find the ID of the word you typed
  if target_word not in word_index:
    return f"Sorry, '{target_word}' is not in the dictionary."

  target_id = word_index[target_word] + 3

  # 4. Grab the 15 mathematical coordinates for that word
  target_vector = embedding_matrix[target_id]

  # 5. Calculate the distance between this word and ALL 88,586 other words
  distances = np.linalg.norm(embedding_matrix - target_vector, axis=1)

  # 6. Sort the distances to find the closest ones
  closest_ids = distances.argsort()[1:top_n+1]

  # 7. Print the results
  print(f"\nWords closest to '{target_word}' in the 15D AI Map:")
  print("-" * 40)
  for i, word_id in enumerate(closest_ids):
    word = reverse_word_index.get(word_id, "<UNKNOWN>")
    dist = distances[word_id]
    print(f"{i+1}. {word} (Distance: {dist:.4f})")

# --- Let's test it out! ---
find_closest_words("excellent")
find_closest_words("terrible")
find_closest_words("boring")


Words closest to 'excellent' in the 15D AI Map:
----------------------------------------
1. favorite (Distance: 0.2720)
2. superb (Distance: 0.2879)
3. edie (Distance: 0.3045)
4. refreshing (Distance: 0.3074)
5. wonderfully (Distance: 0.3095)

Words closest to 'terrible' in the 15D AI Map:
----------------------------------------
1. forgettable (Distance: 0.2057)
2. worse (Distance: 0.2211)
3. fails (Distance: 0.2264)
4. mst3k (Distance: 0.2451)
5. pointless (Distance: 0.2521)

Words closest to 'boring' in the 15D AI Map:
----------------------------------------
1. worse (Distance: 0.3221)
2. disappointment (Distance: 0.3689)
3. pointless (Distance: 0.3842)
4. fails (Distance: 0.3859)
5. poor (Distance: 0.3951)
